<a href="https://colab.research.google.com/github/VickkiMars/Transact/blob/master/cAlvin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
import torch
import json
from transformers import pipeline

In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation",
                model="HuggingFaceH4/zephyr-7b-beta",
                torch_dtype=torch.bfloat16,
                device_map="auto"
                )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [3]:
def format_prompt(query="", messages=False):
  if not messages:
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": query,
        }
    ]
  prompt = pipe.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
  return prompt

In [4]:
user_input = input()

hello, please send 2h to 9032751819


In [22]:
def load_data():
  with open("data.json", 'r') as f:
    data = json.load(f)
  return data

In [23]:
def process_input(input, data):
  data[-1]['content'] = data[-1]['content'].format(input)
  return data

In [24]:
user_input = "hello, please send 9h to 12967482930"
shots = process_input(user_input, load_data())

In [26]:
few_shot_prompt = format_prompt(messages=shots)

In [27]:
outputs = pipe(few_shot_prompt, max_new_tokens=64, do_sample=True, top_p=0.95, temperature=0.01, num_return_sequences=1, return_full_text=False)

In [28]:
res = outputs[0]["generated_text"]

In [29]:
# amount | bank name | account number | name
response = res.split('|')
def process_text(response):
  rs_js = {}
  if len(response) == 4:
    rs_js['amount'] = int(response[0].strip())
    rs_js['bank_name'] = response[1].lower().strip()
    rs_js['account_number'] = response[2].strip()
    rs_js['name'] = response[3].lower().strip()

    assert isinstance(rs_js['amount']) == int
    assert isinstance(rs_js['bank_name']) == str
    assert isinstance(rs_js['account_number']) == int
    assert len(str(rs_js['account_number'])) == 10
    assert isinstance(rs_js['name']) == str

  elif len(response) == 3 and isinstance(response[1]) == str :
    rs_js['amount'] = response[0].strip()
    rs_js['bank_name'] = response[1].lower().strip()
    rs_js['account_number'] = response[2].lower().strip()
  else:
    rs_js['amount'] = response[0].strip()
    rs_js['account_number'] = response[1].lower().strip()

  return rs_js


In [30]:
process_text(response)

{'amount': '9000', 'account_number': '12967482930'}